In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [2]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [3]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [4]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [5]:
# Generate running word count
wordCounts = words.groupBy("word").count()

In [6]:
# Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

#query.awaitTermination()

In [7]:
#nc -lk 9999 

In [8]:
#Submit Job
#unset PYSPARK_DRIVER_PYTHON
#unset PYSPARK_DRIVER_PYTHON_OPTS
#
#$SPARK_HOME/bin/spark-submit $SPARK_HOME/examples/src/main/python/sql/streaming/structured_network_wordcount.py localhost 9999